In [ ]:
# Import Libraries
import csv
import gzip
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import keras
from keras.layers import Activation
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time as time
%matplotlib inline

In [ ]:
# Load (synthetic) Training Data for Neural network
f = gzip.GzipFile('rBergomiTrainSet.txt.gz', "r")
dat=np.load(f)

# Variable xx: rBergomi Parameters
# Variable yy: Implied Volatilities
xx=dat[:,:4]
yy=dat[:,4:]

# Define fixed strike/maturity grid
strikes=np.array([0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5 ])
maturities=np.array([0.1,0.3,0.6,0.9,1.2,1.5,1.8,2.0 ])
strikes_dim=len(strikes)
maturities_dim=len(maturities)
S0=1.

# Define upper and lower bounds for the parameters
ub=0.16,4,-0.1,0.5
lb=[0.01,0.3,-0.95,0.025]

In [ ]:
# Split xx, and yy into training and validation (test) sets
X_train, X_test, y_train, y_test = train_test_split(yy, xx, test_size=0.15, random_state=42)

# Normalize data to help reduce dimensionality
scale = StandardScaler()
scale2=  StandardScaler()
y_train_transform = scale.fit_transform(y_train)
y_test_transform = scale.transform(y_test)
x_train_transform = scale2.fit_transform(X_train)
x_test_transform = scale2.transform(X_test)


def xtransform(X_train,X_test):
    return [scale2.transform(X_train),scale2.transform(X_test)]

def xinversetransform(x):
    return scale2.inverse_transform(x)

def myscale(x):
    res=np.zeros(4)
    for i in range(4):
        res[i]=(x[i] - (ub[i] + lb[i])*0.5) * 2 / (ub[i] - lb[i])
    return res

def myinverse(x):
    res=np.zeros(4)
    for i in range(4):
        res[i]=x[i]*(ub[i] - lb[i]) *0.5 + (ub[i] + lb[i])*0.5
    return res

y_train_transform = np.array([myscale(y) for y in y_train])
y_test_transform = np.array([myscale(y) for y in y_test])
[x_train_transform,x_test_transform]=xtransform(X_train,X_test)

In [ ]:
##############################################################################
# ----------------- Building the Neural network in Keras---------------------#
##############################################################################
keras.backend.set_floatx('float64')

# Shaping the input layer of size 4, equal to number of rbergomi parameters
input1 = keras.layers.Input(shape=(4,))

# Defining the 3 hidden layers with 30 neurons each, using activation function = elu
x1 = keras.layers.Dense(30,activation = 'elu')(input1)
x2=keras.layers.Dense(30,activation = 'elu')(x1) 
x3=keras.layers.Dense(30,activation = 'elu')(x2) 

# Defining the Output Layer for 11 strikes and 8 maturities = 88 output values
x4=keras.layers.Dense(88,activation = 'linear')(x3)

# Defining the network structure
modelGEN = keras.models.Model(inputs=input1, outputs=x4)
modelGEN.summary()


In [ ]:
##############################################################################
# -------------- Training Step of Neural Network in Keras -------------------#
##############################################################################

# Defining the loss function
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))
    
# Compiling the neural network by defining loss function and optimizer="adam"
modelGEN.compile(loss = root_mean_squared_error, optimizer = "adam")

# Fitting the neural network with batchsize=32; epochs=200
modelGEN.fit(y_train_transform, x_train_transform, batch_size=32,validation_data = (y_test_transform,x_test_transform),epochs = 200, verbose = True,shuffle=1)

In [ ]:
# Saving and/or loading NN weights
modelGEN.save_weights('RoughBergomiNNWeights.h5')
modelGEN.load_weights('RoughBergomiNNWeights.h5')

In [ ]:
##############################################################################
# ----------------- Building the Neural network in Numpy --------------------#
##############################################################################


NumLayers=3

def elu(x):
    #Careful function ovewrites x
    ind=(x<0)
    x[ind]=np.exp(x[ind])-1
    return x

def eluPrime(y):
    # we make a deep copy of input x
    x=np.copy(y)
    ind=(x<0)
    x[ind]=np.exp(x[ind])
    x[~ind]=1
    return x

def NeuralNetwork(x):
    input1=x
    for i in range(NumLayers):
        input1=np.dot(input1,NNParameters[i][0])+NNParameters[i][1]
        #Elu activation
        input1=elu(input1)
    #The output layer is linnear
    i+=1
    return np.dot(input1,NNParameters[i][0])+NNParameters[i][1]

def NeuralNetworkGradient(x):
    input1=x
    #Identity Matrix represents Jacobian with respect to initial parameters
    grad=np.eye(4)
    #Propagate the gradient via chain rule
    for i in range(NumLayers):
        input1=(np.dot(input1,NNParameters[i][0])+NNParameters[i][1])
        grad=(np.einsum('ij,jk->ik', grad, NNParameters[i][0]))
        #Elu activation
        grad*=eluPrime(input1)
        input1=elu(input1)
    #input1.append(np.dot(input1[i],NNParameters[i+1][0])+NNParameters[i+1][1])
    grad=np.einsum('ij,jk->ik',grad,NNParameters[i+1][0])
    #grad stores all intermediate Jacobians, however only the last one is used here as output
    return grad

def plot_func(xi0,nu,rho,H):
    x0=myscale(np.array([xi0,nu,rho,H]))
    Smiles=xinversetransform(NeuralNetwork(x0))
    plt.figure(1,figsize=(14,12))
    for i in range(8):
        plt.subplot(4,4,i+1)
        plt.plot(np.log(strikes/1),Smiles[i*strikes_dim:(i+1)*strikes_dim],'*b',label=" NN Approx")
        plt.ylim(0.1,0.8)
        plt.title("Maturity=%1.2f "%maturities[i])
        plt.xlabel("log-moneyness")
        plt.ylabel("Implied vol")
        plt.legend()
    plt.tight_layout()

In [ ]:
##############################################################################
# --------------- Interactive Plotting using the numpy NN -------------------#
##############################################################################

interact(plot_func, xi0 = widgets.FloatSlider(value=0.04,
                                               min=0.01,
                                               max=0.16,
                                               step=0.01),
        nu = widgets.FloatSlider(value=2,
                                               min=0.3,
                                               max=4.0,
                                               step=0.1),
        rho = widgets.FloatSlider(value=-0.7,
                                               min=-0.95,
                                               max=-0.1,
                                               step=0.05),
        H = widgets.FloatSlider(value=0.1,
                                               min=0.025,
                                               max=0.5,
                                               step=0.05))

In [ ]:


strikeslabel=np.round(np.linspace(strikes[0],strikes[-1],8),1)
maturitieslabel=np.array([0.1,0.2, 0.6, 1.5,1.8])
##### AVERAGE VALUES #######
X_sample = xinversetransform(x_test_transform)
y_sample = y_test_transform

prediction=[xinversetransform(modelGEN.predict(y_sample[i].reshape(1,4))[0]) for i in range(len(y_sample))]
plt.figure(1,figsize=(14,4))
ax=plt.subplot(1,3,1)
err = np.mean(100*np.abs((X_sample-prediction)/X_sample),axis = 0)
plt.title("Average relative error",fontsize=15,y=1.04)
plt.imshow(err.reshape(maturities_dim,strikes_dim))
plt.colorbar(format=mtick.PercentFormatter())

ax.set_xticks(np.linspace(0,len(strikes)-1,len(strikes)))
ax.set_xticklabels(strikes)
ax.set_yticks(np.linspace(0,len(maturities)-1,len(maturities)))
ax.set_yticklabels(maturities)
plt.xlabel("Strike",fontsize=15,labelpad=5)
plt.ylabel("Maturity",fontsize=15,labelpad=5)

ax=plt.subplot(1,3,2)
err = 100*np.std(np.abs((X_sample-prediction)/X_sample),axis = 0)
plt.title("Std relative error",fontsize=15,y=1.04)
plt.imshow(err.reshape(maturities_dim,strikes_dim))
plt.colorbar(format=mtick.PercentFormatter())
ax.set_xticks(np.linspace(0,len(strikes)-1,len(strikes)))
ax.set_xticklabels(strikes)
ax.set_yticks(np.linspace(0,len(maturities)-1,len(maturities)))
ax.set_yticklabels(maturities)
plt.xlabel("Strike",fontsize=15,labelpad=5)
plt.ylabel("Maturity",fontsize=15,labelpad=5)

ax=plt.subplot(1,3,3)
err = 100*np.max(np.abs((X_sample-prediction)/X_sample),axis = 0)
plt.title("Maximum relative error",fontsize=15,y=1.04)
plt.imshow(err.reshape(maturities_dim,strikes_dim))
plt.colorbar(format=mtick.PercentFormatter())
ax.set_xticks(np.linspace(0,len(strikes)-1,len(strikes)))
ax.set_xticklabels(strikes)
ax.set_yticks(np.linspace(0,len(maturities)-1,len(maturities)))
ax.set_yticklabels(maturities)
plt.xlabel("Strike",fontsize=15,labelpad=5)
plt.ylabel("Maturity",fontsize=15,labelpad=5)
plt.tight_layout()
plt.savefig('rBergomiNNErrors.png', dpi=300)
plt.show()